In [17]:
#We import everything needed:
import numpy as np
import scipy.stats as sts
import requests
import matplotlib.pyplot as mlab
import matplotlib.pyplot as plt
import math as math
import pandas as pd
from pandas import DataFrame, Series
from scipy.stats import gamma
import scipy.special as spc

In [18]:
#Open and load data:

data = pd.read_csv('MacroSeries.txt', sep=",", header=None, names=["c","k","w","r"]) 
#We put it names to the column so it is easier to call them later


#Question 2a

In [19]:
#First we compute (back) "z" using eqn (3). Note that it depends on w, k and alfa:
def z_back_3(w,k,alfa):
    z = np.log(w/(((k)**alfa)*(1-alfa)))
    return z

#We define a criterion function for "z" knowing that it follows a normal distribution
def crit_z3(params,data):
    alfa, rho, mu, sigma = params
    z = z_back_3(data['w'],data['k'],alfa)
    z_lag = z.shift(1)
    z_lag[0] = mu   #According to whatis given in the question
    
    #"z" follows a normal distribution. We are computing the log-likelihood:
    log_lik_val = - np.log(sigma) - 0.5*np.log(2 * np.pi) - (z - (rho*z_lag+(1-rho)*mu))**2 / (2 * sigma**2)
    
    return -np.mean(log_lik_val)

In [20]:
#MLE estimation of parameters:
import scipy.optimize as opt

#We begin with some guess for the initial values. Unlike Question 1, we are not given any suggestion to where to start the
#values so we arbitrary choose them. (Note: Changing the initial values changes the optimized values)
alfa_init = 0.1
rho_init = 0.1
mu_init = 0.1
sigma_init = 1
params_init = np.array([alfa_init, rho_init, mu_init, sigma_init])
mle_args = data

#We use the 'L_BFGS-B' method because is the only one that returns the inverse Hessian. We also put the bound so that
#alfa, rho, mu and sigma are constrained according to what it is said in the question.
results_cstr_a = opt.minimize(crit_z3, params_init, args=(mle_args), method='L-BFGS-B',
                            bounds=((1e-10, 0.9999), (-0.9999, 0.9999),(1e-10, None), (1e-10, None)))
alfa_MLE_a, rho_MLE_a, mu_MLE_a, sigma_MLE_a = results_cstr_a.x
log_lik_val_a = crit_z3([alfa_MLE_a,rho_MLE_a,mu_MLE_a,sigma_MLE_a],data)
inv_hess = results_cstr_a.hess_inv.todense()

print('alfa_MLE_a=', alfa_MLE_a)
print('rho_MLE_a=', rho_MLE_a)
print('mu_MLE_a=', mu_MLE_a)
print('sigma_MLE_a=', sigma_MLE_a)
print('Log-likelihood=', log_lik_val_a)
print('Variance-Covariance matrix\n ', inv_hess)

alfa_MLE_a= 0.45748197895249465
rho_MLE_a= 0.7205109687252601
mu_MLE_a= 9.523194350515697
sigma_MLE_a= 0.09199596967853813
Log-likelihood= -0.9670690801228998
Variance-Covariance matrix
  [[ 2.19901603e+01 -4.40992844e+01 -3.17083758e+02 -7.23370956e-01]
 [-4.40992844e+01  9.64729330e+01  6.38161496e+02  1.64509107e+00]
 [-3.17083758e+02  6.38161496e+02  4.57294356e+03  1.04907487e+01]
 [-7.23370956e-01  1.64509107e+00  1.04907487e+01  3.47710374e-02]]


Question 2b

In [21]:
#Now we compute (back) "z" using eqn (4). Note that it depends on r, k and alfa
def z_back_4(r,k,alfa):
    z = np.log(r/(((k)**(alfa-1))*(alfa)))
    return z

#Criterion function
def crit_z4(params,data):
    alfa, rho, mu, sigma = params
    z = z_back_4(data['r'],data['k'],alfa)
    z_lag = z.shift(1)
    z_lag[0] = mu      #According to whatis given in the question
    
    #"z" follows a normal distribution. We are computing the log-likelihood::
    log_lik_val = - np.log(sigma) - 0.5*np.log(2 * np.pi) - (z - (rho*z_lag+(1-rho)*mu))**2 / (2 * sigma**2)
    
    return -np.mean(log_lik_val)

In [12]:
#MLE estimation of parameters:
import scipy.optimize as opt

#We begin with some guess for the initial values. We will use the same used in part (a). We could also use, for example, the
#MLE estimates from part (a). (Note: Changing the initial values changes the optimized values)
alfa_init = 0.1
rho_init = 0.1
mu_init = 0.1
sigma_init = 1
params_init = np.array([alfa_init, rho_init, mu_init, sigma_init])
mle_args = data

#We use the 'L_BFGS-B' method because is the only one that returns the inverse Hessian. We also put the bound so that
#alfa, rho, mu and sigma are constrained according to what it is said in the question.
results_cstr_b = opt.minimize(crit_z4, params_init, args=(mle_args), method='L-BFGS-B',
                            bounds=((1e-10, 0.9999), (-0.9999, 0.9999),(1e-10, None), (1e-10, None)))
alfa_MLE_b, rho_MLE_b, mu_MLE_b, sigma_MLE_b = results_cstr_b.x
log_lik_val_b = crit_z4([alfa_MLE_b,rho_MLE_b,mu_MLE_b,sigma_MLE_b],data)
inv_hess_b = results_cstr_b.hess_inv.todense()

print('alfa_MLE_b=', alfa_MLE_b)
print('rho_MLE_b=', rho_MLE_b)
print('mu_MLE_b=', mu_MLE_b)
print('sigma_MLE_b=', sigma_MLE_b)
print('Log-likelihood=', log_lik_val_b)
print('Variance-Covariance matrix\n ', inv_hess_b)

alfa_MLE_b= 0.7018970334618823
rho_MLE_b= 0.4800169691930428
mu_MLE_b= 5.077378123134153
sigma_MLE_b= 0.09204404228038665
Log-likelihood= -0.9665373698056341
Variance-Covariance matrix
  [[ 7.01342559e+01 -8.61010410e+01 -1.20605820e+03  7.19879234e+00]
 [-8.61010410e+01  2.39903561e+02  1.47228032e+03 -6.66048300e+00]
 [-1.20605820e+03  1.47228032e+03  2.07404438e+04 -1.23911369e+02]
 [ 7.19879234e+00 -6.66048300e+00 -1.23911369e+02  8.33811000e-01]]


Question 2c

In [16]:
#Initial values for 'r' and 'k'
r = 1
k = 7500000

#We compute 'z*' according to eqn (4). It is the same formula que use in part (b). We name the this variable z_target
#We need to employ a value for alfa. Since we are using the back out procedure using eqn (4), we decide to use the ML 
#estimates of alfa from part (b)
z_target = np.log(r/(((k)**(alfa_MLE_b-1))*(alfa_MLE_b)))
print('z* is ',z_target)

#Computing the probability r > 1 is equivalent to computting the probability that z > z*. Moreover we know the mean and
#sigma of z, so we can compute the proability using the CDF of a normal distribution. Note that z_t-1 = 10:
prob = 1 - sts.norm.cdf(z_target, loc = rho_MLE_b*10 + (1-rho_MLE_b)*mu_MLE_b, scale = sigma_MLE_b)
print('Probability r>1 is ',prob)

z* is  5.073061810972623
Probability r>1 is  1.0
